In [1]:
import os
import sys

spark_path = "/opt/spark/"

os.environ['SPARK_HOME'] = spark_path
os.environ['HADOOP_HOME'] = spark_path
os.environ['SPARK_CLASSPATH'] = "/home/ubuntu/postgresql-9.3-1103-jdbc41.jar"

sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.3-src.zip")

In [2]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext, Row

In [3]:
sc = SparkContext("local", "test")

In [4]:
sc

In [5]:
sqlctx = SQLContext(sc)

In [6]:
sqlctx

In [7]:
properties = {
    "user": "clientdemo",
    "password": "clientdemo4"
}

In [8]:
url="jdbc:postgresql://postgresdemo.c3yxphqgag3s.ap-southeast-1.rds.amazonaws.com:5432/postgresdemo"

In [9]:
df = sqlctx.read.jdbc(url=url, table="trans", properties=properties)

In [10]:
df.take(5)

[Row(userid=u'3893506', quantity=0, product_id=u'com.gopaktor.subscription.v1.premium.1m', purchase_date=datetime.datetime(2016, 9, 7, 13, 59, 45), platform=u'ios', revenue=31.28, renewal=0, insider=0, receipt_id=u'1984955'),
 Row(userid=u'3793254', quantity=0, product_id=u'com.gopaktor.subscription.v1.premium.1m', purchase_date=datetime.datetime(2016, 9, 25, 3, 6, 11), platform=u'ios', revenue=31.28, renewal=0, insider=0, receipt_id=u'2170195'),
 Row(userid=u'6148794', quantity=0, product_id=u'com.gopaktor.subscription.v1.premium.1m', purchase_date=datetime.datetime(2016, 10, 1, 2, 27, 37), platform=u'ios', revenue=31.28, renewal=0, insider=0, receipt_id=u'2259995'),
 Row(userid=u'6149220', quantity=0, product_id=u'com.gopaktor.subscription.v4.premium.1m', purchase_date=datetime.datetime(2016, 10, 1, 6, 39, 16), platform=u'android', revenue=31.28, renewal=0, insider=0, receipt_id=u'1980761'),
 Row(userid=u'6149432', quantity=0, product_id=u'com.gopaktor.subscription.v4.premium.1m', pu

In [11]:
df.count()

4428

In [12]:
df.columns

['userid',
 'quantity',
 'product_id',
 'purchase_date',
 'platform',
 'revenue',
 'renewal',
 'insider',
 'receipt_id']

In [13]:
data = df.collect()

In [14]:
import pandas as pd

In [15]:
data_df = pd.DataFrame(data, columns=df.columns)

In [16]:
data_df.head()

,userid,quantity,product_id,purchase_date,platform,revenue,renewal,insider,receipt_id
0,3893506,0,com.gopaktor.subscription.v1.premium.1m,2016-09-07 13:59:45,ios,31.28,0,0,1984955
1,3793254,0,com.gopaktor.subscription.v1.premium.1m,2016-09-25 03:06:11,ios,31.28,0,0,2170195
2,6148794,0,com.gopaktor.subscription.v1.premium.1m,2016-10-01 02:27:37,ios,31.28,0,0,2259995
3,6149220,0,com.gopaktor.subscription.v4.premium.1m,2016-10-01 06:39:16,android,31.28,0,0,1980761
4,6149432,0,com.gopaktor.subscription.v4.premium.1m,2016-10-01 09:07:49,android,31.28,0,0,1980865


In [17]:
import yaml
with open('/home/ubuntu/MappingBuffer.yml', 'r') as f:
    doc = yaml.load(f)

In [18]:
doc['column_map']['TRANSACTION_MASTER']

{'cust_id': 'userid',
 'product_id': 'product_id',
 'quantity': 'quantity',
 'renewal': 'renewal',
 'revenue': 'revenue',
 'timestamp': 'purchase_date'}

In [19]:
data_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['cust_id']:'cust_id'}, inplace = True)
data_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['revenue']:'revenue'}, inplace = True)
data_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['product_id']:'product_id'}, inplace = True)
data_df.rename(columns={doc['column_map']['TRANSACTION_MASTER']['timestamp']:'timestamp'}, inplace = True)

In [20]:
data_df.head()

,cust_id,quantity,product_id,timestamp,platform,revenue,renewal,insider,receipt_id
0,3893506,0,com.gopaktor.subscription.v1.premium.1m,2016-09-07 13:59:45,ios,31.28,0,0,1984955
1,3793254,0,com.gopaktor.subscription.v1.premium.1m,2016-09-25 03:06:11,ios,31.28,0,0,2170195
2,6148794,0,com.gopaktor.subscription.v1.premium.1m,2016-10-01 02:27:37,ios,31.28,0,0,2259995
3,6149220,0,com.gopaktor.subscription.v4.premium.1m,2016-10-01 06:39:16,android,31.28,0,0,1980761
4,6149432,0,com.gopaktor.subscription.v4.premium.1m,2016-10-01 09:07:49,android,31.28,0,0,1980865


In [21]:
product = data_df.product_id.unique()

In [22]:
len(product)

18

In [23]:
cust = data_df.cust_id.unique()

In [24]:
len(cust)

3087

In [25]:
cust_prod = data_df[['cust_id','product_id']]

In [26]:
cust_prod.head()

,cust_id,product_id
0,3893506,com.gopaktor.subscription.v1.premium.1m
1,3793254,com.gopaktor.subscription.v1.premium.1m
2,6148794,com.gopaktor.subscription.v1.premium.1m
3,6149220,com.gopaktor.subscription.v4.premium.1m
4,6149432,com.gopaktor.subscription.v4.premium.1m


In [27]:
values = pd.DataFrame([1]*len(cust_prod))

In [28]:
values.rename(columns={0:'rating'}, inplace=True)

In [29]:
cust_prod_df = pd.concat([cust_prod,values],axis=1)

In [30]:
cust_prod_df.head()

,cust_id,product_id,rating
0,3893506,com.gopaktor.subscription.v1.premium.1m,1
1,3793254,com.gopaktor.subscription.v1.premium.1m,1
2,6148794,com.gopaktor.subscription.v1.premium.1m,1
3,6149220,com.gopaktor.subscription.v4.premium.1m,1
4,6149432,com.gopaktor.subscription.v4.premium.1m,1


In [31]:
cust_prod_df['product_id'] = cust_prod_df['product_id'].astype('category')

In [32]:
cust_prod_df['product_id'] = cust_prod_df['product_id'].cat.codes

In [33]:
cust_prod_df['cust_id'] = cust_prod_df['cust_id'].astype('category')

In [34]:
cust_prod_df['cust_id'] = cust_prod_df['cust_id'].cat.codes

In [35]:
cust_prod_df.head(6)

,cust_id,product_id,rating
0,7,5,1
1,6,5,1
2,10,5,1
3,15,17,1
4,21,17,1
5,22,17,1


In [37]:
cust_prod_df.set_index('cust_id', drop=True, inplace = True)

In [38]:
dictionary = cust_prod_df.to_dict(orient="index")

In [39]:
dictionary

{0: {'product_id': 5, 'rating': 1},
 1: {'product_id': 17, 'rating': 1},
 2: {'product_id': 5, 'rating': 1},
 3: {'product_id': 14, 'rating': 1},
 4: {'product_id': 7, 'rating': 1},
 5: {'product_id': 7, 'rating': 1},
 6: {'product_id': 5, 'rating': 1},
 7: {'product_id': 5, 'rating': 1},
 8: {'product_id': 9, 'rating': 1},
 9: {'product_id': 9, 'rating': 1},
 10: {'product_id': 5, 'rating': 1},
 11: {'product_id': 17, 'rating': 1},
 12: {'product_id': 17, 'rating': 1},
 13: {'product_id': 17, 'rating': 1},
 14: {'product_id': 5, 'rating': 1},
 15: {'product_id': 17, 'rating': 1},
 16: {'product_id': 0, 'rating': 1},
 17: {'product_id': 17, 'rating': 1},
 18: {'product_id': 17, 'rating': 1},
 19: {'product_id': 17, 'rating': 1},
 20: {'product_id': 17, 'rating': 1},
 21: {'product_id': 17, 'rating': 1},
 22: {'product_id': 17, 'rating': 1},
 23: {'product_id': 5, 'rating': 1},
 24: {'product_id': 17, 'rating': 1},
 25: {'product_id': 17, 'rating': 1},
 26: {'product_id': 17, 'rating': 